In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os

In [84]:
path = 'data'
#os.chdir(path)
df = pd.read_csv('working_data.csv')
df.head()

,album name,track name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_0,Clusters
0,ASCEND (Tour Edits),Good Things Fall Apart vs. Sad Songs (With Sai...,52,0.563,0.555,2,-5.429,1,0.0288,0.110,0.000001,0.108,0.267,144.336,366253,4,ILLENIUM,2
1,Perfect Way To Die,Perfect Way To Die,30,0.365,0.398,5,-5.695,1,0.0530,0.934,0.000244,0.209,0.259,84.445,211621,4,Alicia Keys,0
2,Bigger Love,"U Move, I Move (feat. Jhené Aiko)",34,0.379,0.249,11,-9.530,1,0.0761,0.840,0.000000,0.107,0.264,83.258,223933,4,John Legend,0
3,Don't Let Me Down (Sabrina Claudio vs. Devault...,Don't Let Me Down - Sabrina Claudio vs. Devaul...,62,0.648,0.651,1,-8.920,1,0.0995,0.298,0.026200,0.252,0.232,107.989,245000,4,Sabrina Claudio vs.,2
4,Quarantine Queen,Quarantine Queen,57,0.485,0.517,9,-4.850,1,0.3070,0.360,0.000000,0.199,0.358,116.641,236307,4,Sinead Harnett,0


**Since there are not many means to tune results from the KMeans model, I will attempt different scaling methods as well as the neighboring tree algorithms (kd tree, ball tree) in order to see how / if results are affected. For comparison's sake, I will choose 3 tracks to represent recommendations for 3 distinct genres**

**MinMax Scaling**

In [3]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

df_x = df.drop(['album name','track name','artist_0'], axis =1)

minmax_x = min_max_scaler.fit_transform(df_x)

In [20]:
from sklearn.neighbors import NearestNeighbors
nbrs_m = NearestNeighbors(n_neighbors = 10).fit(minmax_x)
distance_m,indices_m = nbrs_m.kneighbors(minmax_x,return_distance=True)

In [46]:
def get_index_from_name(name):
    return df[df['track name']==name].index.tolist()[0]

all_track_names = list(df['track name'].values)
def get_id_from_partial_name(partial):
    for name in all_track_names:
        if partial in name:
            print(name,all_track_names.index(name))

def print_similar_tracks_minmax(query = None,id = None):
    if id!=None:
        for i in indices_m[id][1:]:
            print(df.iloc[i]['artist_0'],"-",df.iloc[i]['track name'])
    if query!=None:
        found_id = get_index_from_name(query)
        for id in indices_m[found_id][1:]:
            print(df.iloc[id]['artist_0'],"-",df.iloc[id]['track name'],"-",distance_m[id][1:])

In [74]:
#Rock Bottom (Generic Pop)
print_similar_tracks_minmax(id=580)

The Lumineers - Cleopatra
DEAN - D (Half Moon)
Twin Shadow - Saturdays (feat. HAIM)
Aries - CAROUSEL
Calvin Harris - I Need Your Love (feat. Ellie Goulding)
Fall Out Boy - Wilson (Expensive Mistakes)
NOTD - So Close
Lola Marsh - You're Mine
Just A Gent - Limelight (feat. R O Z E S)


In [23]:
#'Til Tonight (EDM)
print_similar_tracks_minmax(id=27)

Lola Marsh - You're Mine
Above & Beyond Group Therapy - It's Yours (Flashback) [ABGT341]
CHVRCHES - Miracle - The Juan Maclean Remix
Black Caviar - Moonpie
courtship. - Stop for Nothing
Kevin George - High Like This
Jauz - Gassed Up
Al Bairre - Let's Fall in Love Some More
RAC - Let Go


In [80]:
#65th & Ingleside (Hip hop / Rap)
print_similar_tracks_minmax(id=342)

Snakehips - All My Friends (feat. Tinashe & Chance the Rapper)
Kygo - Remind Me to Forget
KAYTRANADA - YOU'RE THE ONE
Kanye West - Gone
Sabrina Claudio vs. - Don't Let Me Down - Sabrina Claudio vs. Devault Remix
Jai Wolf - Lost (feat. Chelsea Jade)
Louis The Child - Better Not (feat. Wafia) - Hotel Garuda Remix
Louis The Child - Free (with Drew Love)
Bleachers - Everybody Lost Somebody


**Absolute scaling**

In [37]:
from sklearn.preprocessing import MaxAbsScaler

maxabs_scaler = MaxAbsScaler()
maxabs_x = maxabs_scaler.fit_transform(df_x)

nbrs_max = NearestNeighbors(n_neighbors = 10).fit(maxabs_x)
distance_max,indices_max = nbrs_max.kneighbors(maxabs_x)

def print_similar_tracks_max(query = None,id = None):
    if id!=None:
        for i in indices_max[id][1:]:
            print(df.iloc[i]['artist_0'],"-",df.iloc[i]['track name'])
    if query!=None:
        found_id = get_index_from_name(query)
        for id in indices_max[found_id][1:]:
            print(df.iloc[id]['artist_0'],"-",df.iloc[id]['track name'])

In [78]:
#Rock Bottom (Generic Pop)
print_similar_tracks_max(id=580)

The Lumineers - Cleopatra
DEAN - D (Half Moon)
Twin Shadow - Saturdays (feat. HAIM)
Calvin Harris - I Need Your Love (feat. Ellie Goulding)
Aries - CAROUSEL
NOTD - So Close
Fall Out Boy - Wilson (Expensive Mistakes)
Mura Masa - 1 Night (feat. Charli XCX)
Just A Gent - Limelight (feat. R O Z E S)


In [31]:
#'Til Tonight (EDM example)
print_similar_tracks_max(id=27)

Black Caviar - Moonpie
Kevin George - High Like This
Lola Marsh - You're Mine
Above & Beyond Group Therapy - It's Yours (Flashback) [ABGT341]
Jauz - Gassed Up
courtship. - Stop for Nothing
CHVRCHES - Miracle - The Juan Maclean Remix
Al Bairre - Let's Fall in Love Some More
Delta Spirit - Bushwick Blues


In [75]:
#65th & Ingleside (hip hop / rap)
print_similar_tracks_max(id=342)

Snakehips - All My Friends (feat. Tinashe & Chance the Rapper)
Bleachers - Everybody Lost Somebody
Kygo - Remind Me to Forget
KAYTRANADA - YOU'RE THE ONE
Sabrina Claudio vs. - Don't Let Me Down - Sabrina Claudio vs. Devault Remix
Louis Futon - Surreal
Kanye West - Gone
Jai Wolf - Lost (feat. Chelsea Jade)
Louis The Child - Free (with Drew Love)


**Standard Scaling**

In [58]:
from sklearn.preprocessing import StandardScaler

standard_scale = StandardScaler()
standard_x = standard_scale.fit_transform(df_x)

nbrs_ss = NearestNeighbors(n_neighbors = 10).fit(standard_x)
distance_ss,indices_ss = nbrs_ss.kneighbors(standard_x)

def print_similar_tracks_ss(query = None,id = None):
    if id!=None:
        for i in indices_ss[id][1:]:
            print(df.iloc[i]['artist_0'],"-",df.iloc[i]['track name'])
    if query!=None:
        found_id = get_index_from_name(query)
        for id in indices_ss[found_id][1:]:
            print(df.iloc[id]['artist_0'],"-",df.iloc[id]['track name'])

In [72]:
#Rock Bottom (Generic Pop)
print_similar_tracks_ss(id=580)

The Lumineers - Cleopatra - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
Twin Shadow - Saturdays (feat. HAIM) - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
DEAN - D (Half Moon) - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
Fall Out Boy - Wilson (Expensive Mistakes) - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
NOTD - So Close - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
Aries - CAROUSEL - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
Lola Marsh - You're Mine - [1.51412871 1.64646448 1.71906408 1.78621344 1.97072972 1.97119908
 2.05912926 2.08623735 2.14846963]
Mr. Probz - Nothing Really Matters - Afrojack Remix - [1.51412871 1.64646448 1.71906408

In [56]:
#'Til Tonight (EDM)
print_similar_tracks_ss(id=27)

CHVRCHES - Miracle - The Juan Maclean Remix - [0.         2.89707535 3.22424012 3.54855894 3.55582333 3.92413717
 4.04818052 4.04850083 4.12758351 4.1529791 ]
Pierce Fulton - Kuaga - Original Mix - [0.         2.89707535 3.22424012 3.54855894 3.55582333 3.92413717
 4.04818052 4.04850083 4.12758351 4.1529791 ]
Above & Beyond Group Therapy - It's Yours (Flashback) [ABGT341] - [0.         2.89707535 3.22424012 3.54855894 3.55582333 3.92413717
 4.04818052 4.04850083 4.12758351 4.1529791 ]
Lola Marsh - You're Mine - [0.         2.89707535 3.22424012 3.54855894 3.55582333 3.92413717
 4.04818052 4.04850083 4.12758351 4.1529791 ]
courtship. - Stop for Nothing - [0.         2.89707535 3.22424012 3.54855894 3.55582333 3.92413717
 4.04818052 4.04850083 4.12758351 4.1529791 ]
ODESZA - Falls - The Knocks Get Up Mix - [0.         2.89707535 3.22424012 3.54855894 3.55582333 3.92413717
 4.04818052 4.04850083 4.12758351 4.1529791 ]
Steve Aoki - Back 2 U (feat. WALK THE MOON) - [0.         2.89707535 3.

In [79]:
#65th & Ingleside (hip hop / rap)
print_similar_tracks_ss(id=342)

KAYTRANADA - YOU'RE THE ONE - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
Kanye West - Gone - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
Various Artists - Quicksand - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
Snakehips - All My Friends (feat. Tinashe & Chance the Rapper) - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
Kendrick Lamar - X (with 2 Chainz & Saudi) - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
Kygo - Remind Me to Forget - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
Louis The Child - Better Not (feat. Wafia) - Hotel Garuda Remix - [2.43937898 2.53217701 2.64325184 2.82378817 2.87486127 3.37124033
 3.45750885 3.48182518 3.50669552]
S

**Personally, I liked the standard scaling recommendations best both in terms of the track choices and ordering (distance from near to far from the initial track). In terms of overall quality in making sensible recommendations I believe the standard scaler provides the best results**

**I will also attempt different neighboring algorithms as a final means to tune the recommendations. The current algorithm is the default which means either of the KD Tree or the Ball Tree algorithms may produce similar results.**

In [94]:
nbrs_bss = NearestNeighbors(n_neighbors = 10,algorithm='brute').fit(standard_x)
distance_bss,indices_bss = nbrs_bss.kneighbors(standard_x)

def print_similar_tracks_bss(query = None,id = None):
    if id!=None:
        for i in indices_bss[id][1:]:
            print(df.iloc[i]['artist_0'],"-",df.iloc[i]['track name'])
    if query!=None:
        found_id = get_index_from_name(query)
        for id in indices_bss[found_id][1:]:
            print(df.iloc[id]['artist_0'],"-",df.iloc[id]['track name'])

In [95]:
#Rock Bottom (Generic Pop)
print_similar_tracks_bss(id=27)

CHVRCHES - Miracle - The Juan Maclean Remix
Pierce Fulton - Kuaga - Original Mix
Above & Beyond Group Therapy - It's Yours (Flashback) [ABGT341]
Lola Marsh - You're Mine
courtship. - Stop for Nothing
ODESZA - Falls - The Knocks Get Up Mix
Steve Aoki - Back 2 U (feat. WALK THE MOON)
RAC - Let Go
Mat Zo - Lust


**It appears that this is indeed the same as the previous results thus the best algorithm determined by the library was ball tree**

In [89]:
nbrs_kss = NearestNeighbors(n_neighbors = 10,algorithm='kd_tree').fit(standard_x)
distance_kss,indices_kss = nbrs_kss.kneighbors(standard_x)

def print_similar_tracks_kss(query = None,id = None):
    if id!=None:
        for i in indices_kss[id][1:]:
            print(df.iloc[i]['artist_0'],"-",df.iloc[i]['track name'])
    if query!=None:
        found_id = get_index_from_name(query)
        for id in indices_kss[found_id][1:]:
            print(df.iloc[id]['artist_0'],"-",df.iloc[id]['track name'])

In [92]:
#Rock Bottom (Generic Pop)
print_similar_tracks_kss(id=27)

CHVRCHES - Miracle - The Juan Maclean Remix
Pierce Fulton - Kuaga - Original Mix
Above & Beyond Group Therapy - It's Yours (Flashback) [ABGT341]
Lola Marsh - You're Mine
courtship. - Stop for Nothing
ODESZA - Falls - The Knocks Get Up Mix
Steve Aoki - Back 2 U (feat. WALK THE MOON)
RAC - Let Go
Mat Zo - Lust


**Results are exactly the same thus choice of algorithms appear to be irrelevant**